# Fraud Losses Data Creation

The Fraud Losses Data Creation script is a series of SQL queries that is executed. These queries are run each Friday at 5 in the morning via a databricks scheduled job so analysts can come to work with the data already ready to be reported on.


Due to copyright reasons, the code has been largely modified and simplified so that code is vague and not revealing of corporate information. However, my hope is the logic and planned structure of the Capital One's First Party Fraud Monthly Business Report Repository is communicated.


## Script Outline

The script is organized as follows:

        1.Set-Up (imports, connections, creating variables)
        2.Writing SQL queries
        3.Running SQL queries
        4.Granting privledge to newly created tables


## Set-Up Explanation

In order to successfully run this script there are a number of processes that must be done in order to connect to the data and run code. They are

        Running the credentials file
        Running utility scripts
        Install the Capital One built package pptmaker
        Importing packages
        Creating useful variables

In [2]:
#Step 1, run credentials files to connect to Capital One's Data infrastructure
%run "Users/[EID]/creds"

#If you are cloning this repository you will have to change the above to speciy your EID

ERROR:root:File `'Users/[EID]/creds.py'` not found.


In [ ]:
#Step 2, run helpful utility scripts that predefine functions used throughout the script
%run "./Utilities/fraud_helper_fx"

In [ ]:
%run "./Utilities/MBR_fx"

In [ ]:
#Step 3, install Capital One internally created package that can create a .pptx file of graphs/tables
dbutils.library.installPyPi("pptmaker", repo='....')

In [2]:
#Step 4, import packages and create helpful variables

from pptmaker import pptMaker
import pyspark.sql.functions as F
from pyspark.sql import DataFrameStatFunctions as FS
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re
import json
import pytz
import os.path
from pytz import timezone

#name developers and recipients -- change this if you are cloning the repository

dev_email = ['joby.george@capitalone.com']
recipients = ['joby.george@capitalone.com']

#set timezone to EST 
tz = pytz.timezone('America/New_York')




In [ ]:
#set up connection to snowflake so we can access productionized data
snowflake_source_name = "net.snowflake.spark.snowflake"
sfOptions = {
    "sfUrl":"...",
    "sfUser":username, #accessed from running creds file
    "sfPassword":password,#accessed from running creds file
    "sfDatabase":"...",
    "sfSchema":"USER_{}".format(username),
}

Utils = spark.jvm.net.snowflake.spark.snowflake.Utils



## Writing SQL Queries

### Note all code is highly simplified to avoid disclosing confidential information

Our goal is to have granular and aggregated data tables containing losses (both fraud and credit losses), where we can easily categorize which losses and accounts are first party fraud. 

To do this, we create a table with the months of data that we'll pull from.

After that we pull from the losses table to see all accounts that have charged-off.

From there we classify these accounts into segments and age buckets.

With data that can be aggregated, we build tables that mimic the monthly reporting of the monthly business report

In [ ]:
#create a table of dates that we will reference
# we want the tables to include everything in the past two years until the 1st of the current month
date_tab = '''
create or replace table lab_fpf.date_tab as (
    select
        date_trunc('month', dateadd(month, -1, current_date)) as max_date
        ,date_trunc('month', dateadd(month, -24, current_date)) as min_date
    );'''



In [ ]:
#losses on an account level
losses_table = '''
create or replace table lab_fpf.losses_base as (
    select
        acct_id
        ,balance as losses
        ,credit_limit
        ,chrgof_dt as chargeoff_date
        ,date_trunc('month', chrgof_dt) as chargeoff_month
        ,open_dt
        ,datediff(day, open_dt, chrgof_dt) as age
        ,fraud_indicator
    from
    chrgof_table
    where snap_dt between (select min_date from lab_fpf.date_tab) and (select max_date from lab_fpf.date_tab)
    );'''



In [ ]:
#classifiying the account into segments
losses_base_segment = '''
create or replace table lab_fpf.loses_segment as (
    select a.*,
           , b.segment 
           , case when age < 365 then 1
           when age between 365 and 730 then 2
           when age between 731 and 1460 then 4
           when age >= 1461 then 9
           end as acct_age_bin
   from lab_fpf.losses_base a
   left join lab_fpf.segments b
   on a.acct_id = b.acct_id 
);'''

In [ ]:
#aggregate the data
losses_agg = '''
create or replace talbe lab_fpf.losses_agg as (
    select
        chargeoff_month
        ,segment
        ,fraud_indicator
        ,acct_age_bin
        ,sum(losses) as total_losses
        , count(*) as num_chargeoffs
    from lab_fpf.losses_segment
    group by 1,2,3,4
    order by 1,2,3,4);'''

In [ ]:
#create specific aggregated segment loss table
#this table is used in the fraud losses graph script
segment_losses_agg = '''
create or repalce table lab_fpf.chart_fraud_loss as (
    select
        chargeoff_month
        ,sum(case when segment = 1 then total_losses else 0 end) as segment_1_losses)
        ,sum(case when segment = 2 then total_losses else 0 end) as segment_2_losses)
        ,sum(case when segment = 3 then total_losses else 0 end) as segment_3_losses)
    from lab_fpf.losses_agg
    where fraud_indicator = 1
    group by 1
    order by 1
        );
        '''

In [ ]:
#creating a table for fraud losses for each age bin, split by segment
young_losses_agg = '''
create or repalce table lab_fpf.chart_fraud_loss_age1 as (
    select
        chargeoff_month
        ,sum(case when segment = 1 then total_losses else 0 end) as segment_1_losses_1)
        ,sum(case when segment = 2 then total_losses else 0 end) as segment_2_losses_1)
        ,sum(case when segment = 3 then total_losses else 0 end) as segment_3_losses_1)
    from lab_fpf.losses_agg
    where fraud_indicator = 1
    and acct_age_bin = 1
    group by 1
    order by 1
        );'''
        
#repeat for accounts aged between 1-2 years
two_year_losses_agg = '''
create or repalce table lab_fpf.chart_fraud_loss_age2 as (
    select
        chargeoff_month
        ,sum(case when segment = 1 then total_losses else 0 end) as segment_1_losses_2)
        ,sum(case when segment = 2 then total_losses else 0 end) as segment_2_losses_2)
        ,sum(case when segment = 3 then total_losses else 0 end) as segment_3_losses_2)
    from lab_fpf.losses_agg
    where fraud_indicator = 1
    and acct_age_bin = 2
    group by 1
    order by 1
        );'''

#repeat for accounts aged between 2-4
middle_age_losses_agg = '''
create or repalce table lab_fpf.chart_fraud_loss_age4 as (
    select
        chargeoff_month
        ,sum(case when segment = 1 then total_losses else 0 end) as segment_1_losses_4)
        ,sum(case when segment = 2 then total_losses else 0 end) as segment_2_losses_4)
        ,sum(case when segment = 3 then total_losses else 0 end) as segment_3_losses_4)
    from lab_fpf.losses_agg
    where fraud_indicator = 1
    and acct_age_bin = 4
    group by 1
    order by 1
        );'''

#repeat for accounts aged more than 4 years
elder_losses_agg = '''
create or repalce table lab_fpf.chart_fraud_loss_age9 as (
    select
        chargeoff_month
        ,sum(case when segment = 1 then total_losses else 0 end) as segment_1_losses_9)
        ,sum(case when segment = 2 then total_losses else 0 end) as segment_2_losses_9)
        ,sum(case when segment = 3 then total_losses else 0 end) as segment_3_losses_9)
    from lab_fpf.losses_agg
    where fraud_indicator = 1
    and acct_age_bin = 9
    group by 1
    order by 1
        );'''

## Running the queries

In order to have databricks run the text queries above we use the 
Utils.runQuery(query) syntax for all of the above queries

In [ ]:
#run date tab
query_list = [date_tab
             ,losses_table
             ,losses_base_segment
             ,losses_agg
             ,segment_losses_agg
             ,young_losses_agg
             ,two_year_losses_agg
             ,middle_age_losses_agg
             ,elder_losses_agg
             ]
for query in query_list:
    Utils.runQuery(query)

## Grant Privledges to the tables

Similarly we just need to Utils.runQuery(''grant select on table to all_users''')

In [ ]:
table_list = ['lab_fpf.date_tab'
              ,'lab_fpf.losses_table'
              ,'lab_fpf.losses_base'
              ,'lab_fpf.losses_agg
              ,'lab_fpf.chart_fraud_loss
              ,'lab_fpf.chart_fraud_loss_age1'
              ,'lab_fpf.chart_fraud_loss_age2'
              ,'lab_fpf.chart_fraud_loss_age4'
              ,'lab_fpf.chart_fraud_loss_age9'
             ]

for table in table_list:
    Utils.runQuery('grant select on ' + table + ' to all_users')